In [37]:
#attempt to modify NTRU Encrypt by using R_p = F_p[S_N] rather than R_p = F_p[C_N] = F_p[x]/(x^N-1)

In [59]:
#the group of units of the symmetric group algebra are computed in Corollary 3.1, 3.2 of
#"Structure of the Unit Group of the Symmetric Group Algebra", Abhilash, Nandakumar

In [2]:
homomorphic_encryption = False

In [1]:
#define the parameters
N=3; p=11; q=41; d=2
assert N.is_prime()
assert p.is_prime()
assert gcd(N,q) == 1 
assert gcd(p,q) == 1
#assert q > (6*d+1)*p

In [4]:
#create the polynomial ring \Z[x]/(x^N-1)
R = SymmetricGroupAlgebra(ZZ,N); R

Symmetric group algebra of order 3 over Integer Ring

In [5]:
#a ternary polynomial in R = \Z[x]/(x^N-1) has d1 coeffs = +1, d2 coeffs = -1
def is_ternary(a,d1,d2):
    assert a in R
    coeffs = [pair[1] for pair in list(a)]
    return list(coeffs).count(+1) == d1 and list(coeffs).count(-1) == d2

In [6]:
#create the symmetric group algebra F_q[S_N]
R_q = SymmetricGroupAlgebra(GF(q),N); R_q

Symmetric group algebra of order 3 over Finite Field of size 41

In [7]:
#create the quotient polynomial ring F_p[S_N]
R_p = SymmetricGroupAlgebra(GF(p),N); R_p

Symmetric group algebra of order 3 over Finite Field of size 11

In [144]:
#create a random unit of the symmetric group algebra over a finite field
#essentially if K_q[S_N] \cong M_{n_1}(K_{q^{m_1}}) \oplus ... \oplus M_{n_r}(K_{q^{m_r}}) then
#\mathcal{U}(K_q[S_N]) \cong GL_{n_1}(K_{q^{m_1}}) \oplus ... \oplus GL_{n_r}(K_{q^{m_r}})
#and the number n_k, m_k can be read off from the Brattelli diagram.
#however, only ~1/p elements are not invertible by determinant considerations, so most elements should be invertible
def random_unit_SGA(p,n,max_iter=1):
    assert p.is_prime()
    for i in range(max_iter):
        random_element = R.random_element()
        try:
            R_p(random_element).inverse()
            return random_element
        except ValueError:
            continue
    print("Could not find invertible element of symmetric group algebra.")

In [151]:
#create an element f of R = \ZZ[S_N] which is invertible in R_p = F_p[S_N]
f = random_unit_SGA(p,N,max_iter=100)
#assert is_ternary(f,d+1,d)
print(f)

-[1, 3, 2] + 5*[2, 3, 1]


In [154]:
#create an element g of R = \ZZ[S_N] which is invertible in R_p = F_p[S_N]
g = random_unit_SGA(p,N,max_iter=100)
#assert is_ternary(g,d,d)
print(g)

2*[1, 3, 2] + [2, 3, 1]


In [155]:
#compute F_p(x) = f(x)^{-1} (mod p)
F_p = R_p(f).inverse(); print(F_p)
assert F_p * R_p(f) == 1

6*[1, 3, 2] + 8*[3, 1, 2]


In [156]:
#compute F_q(x) = f(x)^{-1} (mod q)
F_q = R_q(f).inverse(); print(F_q)
assert F_q * R_q(f) == 1

12*[1, 3, 2] + 19*[3, 1, 2]
